In [8]:
#!pip -q install datasets transformers sentence-transformers scikit-learn tqdm
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 512 kB 918 kB/s eta 0:00:01
     |████████████████████████████████| 133 kB 15.8 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 7.8 MB/s  eta 0:00:01
     |████████████████████████████████| 119 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 31.2 MB 275 kB/s eta 0:00:01
     |████████████████████████████████| 10.8 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 492 kB 116 kB/s eta 0:00:011
     |████████████████████████████████| 47 kB 738 kB/s eta 0:00:01
     |████████████████████████████████| 67 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 50 kB 20.9 MB/s eta 0:00:01
     |████████████████████████████████| 44 kB 12.0 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 9.3 MB/s  eta 0:00:01
     |████████████████████████████████| 113 kB 40.3 MB/s eta 0:00:01
     |███████████████████████████

In [ ]:
from datasets import load_dataset
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

data=load_dataset("rotten_tomatoes")

def evaluate(y_true,y_pred,title=""):
    if title:
        print(title)
        print(classification_report(y_true,y_pred))
        print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))
    else:
        print("No Title provided please provide one")



In [21]:
from transformers import pipeline

MODEL_ID = "cardiffnlp/twitter-roberta-base-sentiment-latest"

pipe=pipeline(
    task='text-classification',
    model=MODEL_ID,
    tokenizer=MODEL_ID,
    top_k=None,
    truncation=True,
    device = "mps"
)

print (pipe.model.config.id2label)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{0: 'negative', 1: 'neutral', 2: 'positive'}


In [26]:
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm


y_pred=[]

for out in tqdm(pipe(KeyDataset(data["test"],"text")), total=len(data["test"])):
    labels=[d["label"].lower() for d in out]
    scores=[d["score"] for d in out]

    pos_score=scores[labels.index("positive")] if "positive" in labels else 0.0
    neg_score=scores[labels.index("negative")] if "negative" in labels else 0.0

    y_pred.append(1 if pos_score >= neg_score else 0)

100%|██████████| 1066/1066 [00:28<00:00, 37.50it/s]


In [27]:
# [7] Evaluate predictions
evaluate(data["test"]["label"], y_pred)


              precision    recall  f1-score   support

           0       0.76      0.88      0.81       533
           1       0.86      0.72      0.78       533

    accuracy                           0.80      1066
   macro avg       0.81      0.80      0.80      1066
weighted avg       0.81      0.80      0.80      1066

Confusion matrix:
 [[469  64]
 [149 384]]


In [1]:
from sentence_transformers import SentenceTransformer

EMBEDDER = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")


/Users/rigvedpatil/Library/Python/3.9/lib/python/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/rigvedpatil/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
W0110 10:17:13.712639 77359 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
/Users/rigvedpatil/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `f

In [ ]:
#. Embed train and test texts

X_train = EMBEDDER.encode(data["train"]["text"], show_progress_bar=True)
X_test = EMBEDDER.encode(data["test"]["text"], show_progress_bar=True)